In [5]:
%pip install pandas==2.1.4
%pip install loguru==0.7.2
%pip install numpy==1.26.4
%pip install scikit-learn==1.3.2
%pip install requests


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached certi

In [6]:
import pandas as pd
import numpy as np
import json
from sklearn.decomposition import PCA

from graph import Step
from language_modeling import OpenAiLlamaApi, LlamaModel, PromptGenerator
from code_generation import ValidationCodeGenerator, MainCodeGenerator
from orchestrator import Orchestrator
from utils import get_dataset_info
from pathlib import Path

In [9]:
EXAMPLE_STEP_SCRIPT = """
import pandas as pd
import pywt
from sklearn.preprocessing import StandardScaler

def step_40(Segments_normalized, Dec_levels):
    Features = []
    for segment in Segments_normalized:
        coeffs = pywt.wavedec(segment, 'db4', level=Dec_levels)
        features = [coefficient.mean() for coefficient in coeffs]
        Features.append(features)
    return StandardScaler().fit_transform(Features)
"""

EXAMPLE_VALIDATION_SCRIPT = """
import pandas as pd
from step_10 import step_10
from step_20 import step_20
from step_30 import step_30
from step_40 import step_40

def validate_step():
    csv_path = '/path/to/your/csv/file.csv'
    raw_data = step_10(csv_path)
    Segments = step_20(raw_data, SizeSegment=512)
    Segments_normalized = step_30(Segments)
    Features = step_40(Segments_normalized, Dec_levels=5)
    print(Features)

if __name__ == '__main__':
    validate_step()
"""

steps = [
    Step(
        step_id="10",
        description="Import raw data from CSV and segment it",
        dependencies=[],
        input_vars=["csv_path", "SizeSegment"],
        output_vars=["Segments"],
        additional_info="Use pandas to read the CSV and create segments of size SizeSegment."
    ),
    Step(
        step_id="20",
        description="Normalize the segmented data using MinMaxScaler",
        dependencies=["10"],
        input_vars=["Segments"],
        output_vars=["Segments_normalized"],
        additional_info="Segments is a list of 1D numpy arrays. Each segment should be normalized independently."
    ),
    Step(
        step_id="30",
        description="Extract features using wavelet decomposition",
        dependencies=["20"],
        input_vars=["Segments_normalized", "Dec_levels"],
        output_vars=["Features"],
        additional_info="Use pywavelets (pywt) library with 'db3' wavelet and specified Dec_levels."
    ),
    Step(
        step_id="40",
        description="Apply PCA for dimension reduction",
        dependencies=["30"],
        input_vars=["Features", "NC_pca"],
        output_vars=["PCA_Features", "pca"],
        additional_info="Use sklearn's PCA. Return both the transformed features and the PCA object."
    ),
    Step(step_id="50",
        description="Train model, evaluate, and calculate metrics",
        dependencies=["40"],
        input_vars=["PCA_Features", "kernel", "nu", "gamma"],
        output_vars=["FittedClassifier", "Prec_learn", "Prec_test"],
        additional_info="""
        1. Create labels: np.ones for learning data.
        2. Split data into train and test sets (80% train, 20% test).
        3. Create and fit a One-Class SVM classifier using sklearn.
        4. Predict labels for training data.
        5. Calculate error rate for training data.
        6. Predict labels for test data (assume all test data as anomaly, i.e., -1).
        7. Calculate error rate for test data.
        8. Calculate precision as 1 - error_rate for both training and test.
        Return the fitted classifier and both precision values.
        """
    )
]

In [10]:
csv_path = str(Path('learning-file_2.csv').resolve())
raw_data = pd.read_csv(csv_path)
dataset_info = get_dataset_info(raw_data)
# Assume raw_data is a pandas DataFrame with 'timestamp' and 'signal' columns
signal_data = raw_data['signal'].values

# Adjust based on data size
SizeSegment = min(512, len(signal_data) // 100)
gamma = 'scale'  # Let sklearn choose an appropriate scale
nu = 0.1  # This might need domain knowledge to set appropriately
kernel = "rbf"  # This is often a good default

# PCA
# We'll use the signal data for PCA parameter calculation
pca = PCA().fit(signal_data.reshape(-1, 1))
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
NC_pca = np.argmax(cumulative_variance_ratio >= 0.95) + 1

Dec_levels = int(np.log2(SizeSegment)) - 3  # Adjust based on segment size

parameters = {
    'csv_path': f"'{csv_path}'",
    "SizeSegment": f"{SizeSegment}",
    "gamma": f"'{gamma}'",
    "nu": f"{nu}",
    "kernel" : f"'{kernel}'",
    "NC_pca": f"{NC_pca}",
    "Dec_levels": f"{Dec_levels}",
}

with open('env.json', 'r') as f:
    credentials_dict = json.load(f)

API_URL = "https://openrouter.ai/api/v1"
API_KEY = credentials_dict["OPENROUTER_API_KEY"]
MODEL_TAG = "meta-llama/llama-3-70b-instruct"
llama_api = OpenAiLlamaApi(API_URL, API_KEY, MODEL_TAG)
model = LlamaModel(llama_api)
prompt_generator = PromptGenerator(EXAMPLE_STEP_SCRIPT, dataset_info)
validation_code_genrator = ValidationCodeGenerator()
main_code_generator = MainCodeGenerator()

orchestrator = Orchestrator(
    model,
    prompt_generator,
    validation_code_genrator,
    main_code_generator,
    'out'
)

In [13]:
orchestrator.run_steps(steps, parameters)

2024-08-08 20:23:06.044 | INFO     | language_modeling:execute_request:28 - Full API response: {'id': 'gen-d4Np0neV6cBQrMBNiImnFvSMjucw', 'model': 'meta-llama/llama-3-70b-instruct', 'object': 'chat.completion', 'created': 1723137781, 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Here is the Python function `step_10`:\n```\nimport pandas as pd\nimport numpy as np\nfrom sklearn.preprocessing import MinMaxScaler\n\ndef step_10(csv_path, SizeSegment):\n    data = pd.read_csv(csv_path)\n    signal_values = data['signal'].values\n    num_segments = int(np.ceil(len(signal_values) / SizeSegment))\n    Segments = [signal_values[i*SizeSegment:(i+1)*SizeSegment] for i in range(num_segments)]\n    Segments_normalized = [MinMaxScaler().fit_transform(segment.reshape(-1, 1)).ravel() for segment in Segments]\n    return Segments_normalized\n```"}, 'finish_reason': 'stop', 'logprobs': {'tokens': None, 'token_logprobs': None, 'top_logprobs': None, 'text_offset': None}}], 'usage':

generated step source for step 10, filename: step_10.py

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def step_10(csv_path, SizeSegment):
    data = pd.read_csv(csv_path)
    signal_values = data['signal'].values
    num_segments = int(np.ceil(len(signal_values) / SizeSegment))
    Segments = [signal_values[i*SizeSegment:(i+1)*SizeSegment] for i in range(num_segments)]
    Segments_normalized = [MinMaxScaler().fit_transform(segment.reshape(-1, 1)).ravel() for segment in Segments]
    return Segments_normalized

generated validation source for step 10, filename: validate_step_10.py
import pandas as pd
from step_10 import step_10


csv_path = '/Users/maxim/Projects/ml/CodeGeneration/src/learning-file_2.csv'
SizeSegment = 307
gamma = 'scale'
nu = 0.1
kernel = 'rbf'
NC_pca = 1
Dec_levels = 5

def validate_step():
    Segments = step_10(csv_path, SizeSegment)
    print(Segments)

if __name__ == '__main__':
    validate_step()


running validate_step

2024-08-08 20:23:15.382 | INFO     | language_modeling:execute_request:28 - Full API response: {'id': 'gen-mDRIAwtM42cWXDWMEjvmoAx4LDX6', 'model': 'meta-llama/llama-3-70b-instruct', 'object': 'chat.completion', 'created': 1723137787, 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Here is the Python function 'step_20' that normalizes the segmented data using MinMaxScaler:\n```\nimport numpy as np\nfrom sklearn.preprocessing import MinMaxScaler\n\ndef step_20(Segments):\n    Segments_normalized = []\n    for segment in Segments:\n        scaler = MinMaxScaler()\n        normalized_segment = scaler.fit_transform(segment.reshape(-1, 1)).flatten()\n        Segments_normalized.append(normalized_segment)\n    return Segments_normalized\n```"}, 'finish_reason': 'stop', 'logprobs': None}], 'usage': {'prompt_tokens': 715, 'completion_tokens': 98, 'total_tokens': 813}}


generated step source for step 20, filename: step_20.py

import numpy as np
from sklearn.preprocessing import MinMaxScaler

def step_20(Segments):
    Segments_normalized = []
    for segment in Segments:
        scaler = MinMaxScaler()
        normalized_segment = scaler.fit_transform(segment.reshape(-1, 1)).flatten()
        Segments_normalized.append(normalized_segment)
    return Segments_normalized

generated validation source for step 20, filename: validate_step_20.py
import pandas as pd
from step_10 import step_10
from step_20 import step_20


csv_path = '/Users/maxim/Projects/ml/CodeGeneration/src/learning-file_2.csv'
SizeSegment = 307
gamma = 'scale'
nu = 0.1
kernel = 'rbf'
NC_pca = 1
Dec_levels = 5

def validate_step():
    Segments = step_10(csv_path, SizeSegment)
    Segments_normalized = step_20(Segments)
    print(Segments_normalized)

if __name__ == '__main__':
    validate_step()


running validate_step_20.py in /Users/maxim/Projects/ml/CodeGeneration/src/out
exit_code 

2024-08-08 20:23:19.302 | INFO     | language_modeling:execute_request:28 - Full API response: {'id': 'gen-dXakTtEzI1OQDtHD1xfQVA12YHrD', 'model': 'meta-llama/llama-3-70b-instruct', 'object': 'chat.completion', 'created': 1723137796, 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Here is the Python function `step_30`:\n```\nimport pywt\nimport numpy as np\n\ndef step_30(Segments_normalized, Dec_levels):\n    Features = []\n    for segment in Segments_normalized:\n        coeffs = pywt.wavedec(segment, 'db3', level=Dec_levels)\n        features = [np.mean(coefficient) for coefficient in coeffs]\n        Features.append(features)\n    return Features\n```"}, 'finish_reason': 'stop', 'logprobs': {'tokens': None, 'token_logprobs': None, 'top_logprobs': None, 'text_offset': None}}], 'usage': {'prompt_tokens': 717, 'completion_tokens': 90, 'total_tokens': 807}}


...

fixing step source for step 50, filename: step_50.py


import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split

def step_50(PC_Features, kernel, nu, gamma):
    labels = np.ones(len(PC_Features))
    X_train, X_test = train_test_split(PC_Features, test_size=0.2, random_state=42)
    
    clf = OneClassSVM(kernel=kernel, nu=nu, gamma=gamma)
    clf.fit(X_train)
    
    y_pred_train = clf.predict(X_train)
    error_rate_train = np.mean(y_pred_train != 1)
    Prec_learn = 1 - error_rate_train
    
    y_pred_test = clf.predict(X_test)
    error_rate_test = np.mean(y_pred_test != 1)
    Prec_test = 1 - error_rate_test
    
    return clf, Prec_learn, Prec_test

running validate_step_50.py in /Users/maxim/Projects/ml/CodeGeneration/src/out
exit_code = 1


2024-08-08 20:24:34.390 | INFO     | language_modeling:execute_request:28 - Full API response: {'id': 'gen-yexnLXoWr2iObcWQG0ZB30VAySIn', 'model': 'meta-llama/llama-3-70b-instruct', 'object': 'chat.completion', 'created': 1723137870, 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Here is the corrected code:\n\n```\nimport numpy as np\nfrom sklearn.svm import OneClassSVM\nfrom sklearn.model_selection import train_test_split\n\ndef step_50(PC_Features, kernel, nu, gamma):\n    labels = np.ones(len(PC_Features))\n    X_train, X_test = train_test_split(PC_Features, test_size=0.2, random_state=42)\n    \n    clf = OneClassSVM(kernel=kernel, nu=nu, gamma=gamma)\n    clf.fit(X_train)\n    \n    y_pred_train = clf.predict(X_train)\n    error_rate_train = np.mean(y_pred_train != 1)\n    Prec_learn = 1 - error_rate_train\n    \n    y_pred_test = clf.predict(X_test)\n    error_rate_test = np.mean(y_pred_test != 1)\n    Prec_test = 1 - error_rate_test\n    \n    return clf, 

fixing step source for step 50, filename: step_50.py


import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split

def step_50(PC_Features, kernel, nu, gamma):
    labels = np.ones(len(PC_Features))
    X_train, X_test = train_test_split(PC_Features, test_size=0.2, random_state=42)
    
    clf = OneClassSVM(kernel=kernel, nu=nu, gamma=gamma)
    clf.fit(X_train)
    
    y_pred_train = clf.predict(X_train)
    error_rate_train = np.mean(y_pred_train != 1)
    Prec_learn = 1 - error_rate_train
    
    y_pred_test = clf.predict(X_test)
    error_rate_test = np.mean(y_pred_test != 1)
    Prec_test = 1 - error_rate_test
    
    return clf, Prec_learn, Prec_test

running validate_step_50.py in /Users/maxim/Projects/ml/CodeGeneration/src/out
exit_code = 1


2024-08-08 20:24:38.608 | INFO     | language_modeling:execute_request:28 - Full API response: {'id': 'gen-IHnA0nB3JRKBDZ6GnzynzSmLpUPa', 'model': 'meta-llama/llama-3-70b-instruct', 'object': 'chat.completion', 'created': 1723137875, 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '```\nimport numpy as np\nfrom sklearn.svm import OneClassSVM\nfrom sklearn.model_selection import train_test_split\n\ndef step_50(PC_Features, kernel, nu, gamma):\n    labels = np.ones(len(PC_Features))\n    X_train, X_test = train_test_split(PC_Features, test_size=0.2, random_state=42)\n    \n    clf = OneClassSVM(kernel=kernel, nu=nu, gamma=gamma)\n    clf.fit(X_train)\n    \n    y_pred_train = clf.predict(X_train)\n    error_rate_train = np.mean(y_pred_train != 1)\n    Prec_learn = 1 - error_rate_train\n    \n    y_pred_test = clf.predict(X_test)\n    error_rate_test = np.mean(y_pred_test != 1)\n    Prec_test = 1 - error_rate_test\n    \n    return clf, Prec_learn, Prec_test\n```'}, '

fixing step source for step 50, filename: step_50.py
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split

def step_50(PC_Features, kernel, nu, gamma):
    labels = np.ones(len(PC_Features))
    X_train, X_test = train_test_split(PC_Features, test_size=0.2, random_state=42)
    
    clf = OneClassSVM(kernel=kernel, nu=nu, gamma=gamma)
    clf.fit(X_train)
    
    y_pred_train = clf.predict(X_train)
    error_rate_train = np.mean(y_pred_train != 1)
    Prec_learn = 1 - error_rate_train
    
    y_pred_test = clf.predict(X_test)
    error_rate_test = np.mean(y_pred_test != 1)
    Prec_test = 1 - error_rate_test
    
    return clf, Prec_learn, Prec_test

running validate_step_50.py in /Users/maxim/Projects/ml/CodeGeneration/src/out
exit_code = 1
